In [1]:
import pandas as pd
import numpy as np
import itertools
import time
import statsmodels.api as sm
import sys
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

df = pd.read_csv('historical_data1_Q12013_clean.csv')
df2 = pd.read_csv('historical_data1_Q22013_clean.csv')

# NOTE: DOES NOT USE mortgage_insurance_perc
dummies = pd.get_dummies(df[['first_time_homebuyer_flag','occupancy_status', 
                             'channel','prepayment_penalty_mortgage_flag',  'loan_purpose']])

y = df.orig_interest_rate
X_ = df.drop(['first_payment_date', 'first_time_homebuyer_flag', 'maturity_date',
              'metropolitan_stat_area', 
              'occupancy_status', 'channel', 'prepayment_penalty_mortgage_flag', 'product_type',
             'property_state', 'property_type', 'postal_code', 'loan_sequence_no',
             'loan_purpose', 'orig_loan_term', 'seller_name', 'service_name',
              'orig_interest_rate','mortgage_insurance_perc'], axis=1).astype('float64')
X = pd.concat([X_, dummies[['first_time_homebuyer_flag_Y', 'occupancy_status_I',
                            'occupancy_status_O', 'occupancy_status_S',
                           'channel_B', 'channel_C', 'channel_R',  'loan_purpose_C',
                           'loan_purpose_N', 'loan_purpose_P']]], axis=1)


# NOTE: DOES NOT USE mortgage_insurance_perc
dummies2 = pd.get_dummies(df2[['first_time_homebuyer_flag','occupancy_status', 
                             'channel','prepayment_penalty_mortgage_flag',  'loan_purpose']])

y_val = df2.orig_interest_rate
X_val_ = df2.drop(['first_payment_date', 'first_time_homebuyer_flag', 'maturity_date',
              'metropolitan_stat_area', 
              'occupancy_status', 'channel', 'prepayment_penalty_mortgage_flag', 'product_type',
             'property_state', 'property_type', 'postal_code', 'loan_sequence_no',
             'loan_purpose', 'orig_loan_term', 'seller_name', 'service_name',
              'orig_interest_rate','mortgage_insurance_perc'], axis=1).astype('float64')
X_val = pd.concat([X_val_, dummies2[['first_time_homebuyer_flag_Y', 'occupancy_status_I',
                            'occupancy_status_O', 'occupancy_status_S',
                           'channel_B', 'channel_C', 'channel_R', 'loan_purpose_C',
                           'loan_purpose_N', 'loan_purpose_P']]], axis=1)

def processSubset(feature_set):
# Fit model on feature_set and calculate RSS
    model = linear_model.LinearRegression()
    model.fit(X[list(feature_set)], y)
    
    #MAE, RMS, MAPE
    RMS_train = mean_squared_error(y, model.predict(X[list(feature_set)]))
    RMS_test = mean_squared_error(y_val, model.predict(X_val[list(feature_set)]))
    
    MAE_train = mean_absolute_error(y, model.predict(X[list(feature_set)]))
    MAE_test = mean_absolute_error(y_val, model.predict(X_val[list(feature_set)]))
    
    MAPE_train = np.mean(np.abs((y - model.predict(X[list(feature_set)])) / y)) * 100
    MAPE_test = np.mean(np.abs((y_val - model.predict(X_val[list(feature_set)])) / y_val)) * 100
    
    
    RSS = ((model.predict(X_val[list(feature_set)]) - y_val) ** 2).sum()
    return {"model":model, "RSS":RSS, "feature":X[list(feature_set)], 
            "RMS_train": RMS_train, "RMS_test": RMS_test,
            "MAE_train": MAE_train, "MAE_test": MAE_test,
            "MAPE_train": MAPE_train, "MAPE_test": MAPE_test}

feature_set = ['orig_loantovalue', 'occupancy_status_I', 'credit_score', 
               'orig_upb', 'loan_purpose_N', 'channel_R', 
               'no_borrower', 'orig_debttoincome', 'occupancy_status_S', 
               'first_time_homebuyer_flag_Y', 
               'loan_purpose_P', 'no_unit', 'occupancy_status_O', 'loan_purpose_C']
model = processSubset(feature_set)


print model["RMS_train"], model["RMS_test"],
print model["MAE_train"], model["MAE_test"],
print model["MAPE_train"], model["MAPE_test"],

/usr/local/lib/python2.7/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


KeyError: "['channel_T' 'prepayment_penalty_mortgage_flag_Y'] not in index"